In [1]:
#importing required models
#import os
#import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import torch
#import torch.nn as nn
#import torch.nn.functional as F
#import torch.optim as optim
#from torchvision import datasets, transforms
#from torch.utils.data import DataLoader
import cv2
#from PIL import Image

### Preparing Data for Deep Learning

In [2]:
path = "speed_images"
for folder in os.listdir(path):
    for img_file in os.listdir(os.path.join(path, folder)):
        img_file = os.path.join(path, folder, img_file)
        
        try:
            img = Image.open(img_file)
            if img.mode != "RGB":
                os.remove(img_file)
        except:
            pass
        

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5]),
])

In [4]:
# creating an image dataset folder for classification
dataset = datasets.ImageFolder("speed_images", transform=transform)
dataset_len = len(dataset)

In [5]:
dataset_len

5034

In [6]:
# Divide dataset into train and test set
train_len, test_len = dataset_len-1000, 1000
train_set, test_set = torch.utils.data.random_split(dataset, [train_len, test_len])
batch_size = 192
len(train_set)

4034

In [7]:
# Create dataloader from train and test sets
train_set = DataLoader(dataset=train_set, shuffle=True, batch_size=batch_size)
test_set = DataLoader(dataset=test_set, shuffle=True, batch_size=batch_size)
len(train_set)

22

### Build Deep Learning Model

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device", device)

using device cpu


In [9]:
learning_rate = 0.001

### Deep Learning Model

In [10]:
class Model(nn.Module): 
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(58320, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [11]:
model = Model().to(device)
print(model)

Model(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=58320, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2, bias=True)
)


In [12]:
# Define loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

### Train model

In [13]:
train_losses = []
for epoch in range(10):
    train_loss = 0.0
    total_correct = 0
    model.train()
    for data, target in train_set:
  
        optimizer.zero_grad()
        #forward-pass
        output = model(data)
        
        output_idx = torch.argmax(output, dim=1)
        total_correct += (target == output_idx).sum().item()
        
        loss = criterion(output, target)
        #backward-pass
        loss.backward()
        # Update the parameters
        optimizer.step()
        # Update the Training loss
        train_loss += loss.item() * data.size(0)
    
    print(f"Epoch: {epoch}, loss = {train_loss/train_len}, Accuracy = {(total_correct/train_len)*100}%")
print("Finished Training")    

Epoch: 0, loss = 0.8163858716240837, Accuracy = 77.59048091224591%
Epoch: 1, loss = 0.16267268144676575, Accuracy = 93.85225582548338%
Epoch: 2, loss = 0.08414873150274264, Accuracy = 96.85176003966288%
Epoch: 3, loss = 0.04015213395595219, Accuracy = 98.63658899355478%
Epoch: 4, loss = 0.02309053451303094, Accuracy = 99.3554784333168%
Epoch: 5, loss = 0.015662948765779682, Accuracy = 99.50421417947447%
Epoch: 6, loss = 0.012841111806230007, Accuracy = 99.55379276152702%
Epoch: 7, loss = 0.009878108078446721, Accuracy = 99.70252850768469%
Epoch: 8, loss = 0.0135528566920645, Accuracy = 99.5785820525533%
Epoch: 9, loss = 0.01198233865035479, Accuracy = 99.5785820525533%
Finished Training


### Test model

In [15]:
with torch.no_grad():
    model.eval()
    total_loss = 0
    total_correct = 0
    for data, target in test_set:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        loss = criterion(outputs, target)
        total_loss += loss.item() * data.size(0)
        output_idx = torch.argmax(outputs, dim=1)
        total_correct += (target == output_idx).sum()
    
    print(f"loss = {train_loss/train_len}, Accuracy = {(total_correct/test_len)*100}%")

loss = 0.01198233865035479, Accuracy = 100.0%


### Save Model

In [18]:
# Save model
model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save("aegis_speed.pt") 

In [19]:
# Load model
model = torch.jit.load("aegis_speed.pt")
model.eval()

RecursiveScriptModule(
  original_name=Model
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (conv2_drop): RecursiveScriptModule(original_name=Dropout2d)
  (fc1): RecursiveScriptModule(original_name=Linear)
  (fc2): RecursiveScriptModule(original_name=Linear)
)

In [20]:
#Function to check if a frame is part of the speed frames 
def check_speed(path):
    img = Image.open(path)
    img = transform(img).unsqueeze(dim=0).to(device)
    prediction = model(img)
    v = prediction.detach().numpy()
    y = v.tolist()
    if y[0][-1] > y[0][0]:
        return True
    return False

In [9]:
video = cv2.VideoCapture("test_videos/Appenzell_18.mp4")
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width

1920

In [10]:
#writer object to write selected frames
writer = cv2.VideoWriter("sum_vid/finalleap91.mp4", cv2.VideoWriter_fourcc(*"DIVX"), 25, (width, height))

In [11]:
ret, frame1 = video.read()
prev_frame = frame1
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
a = []

In [12]:
currentframe = 0
a = 0
b = 0
c = 0



In [13]:
while ret:
    ret, frame = video.read()
    nex = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    if ret is True:
        currentframe += 1
        flow = cv2.calcOpticalFlowFarneback(prvs,nex, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        if np.mean(mag) > 7: #Check if a frame is part of speed frame
            writer.write(frame) #Write the frame into the writer
            prev_frame = frame
            a += int(1)
            
        else:
            prev_frame = frame
            b += 1
            
            
        c += 1
        
        
    #if cv2.waitKey(1) & 0xFF == ord("q"):
       # break
        
print("Total Frames: ", c)
print("Unique Frames: ", a)
print("Common Frames: ", b)
video.release()
writer.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [8]:
while ret:
    ret, frame = video.read()
    if ret is True:
        currentframe += 1
        flow = cv2.calcOpticalFlowFarneback(prvs,nex, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        if np.mean(ang) > 4: #Check if a frame is part of speed frame
            writer.write(frame) #Write the frame into the writer
            prev_frame = frame
            a += int(1)
            
        else:
            prev_frame = frame
            b += 1
            
            
        c += 1
        
        
    #if cv2.waitKey(1) & 0xFF == ord("q"):
       # break
        
print("Total Frames: ", c)
print("Unique Frames: ", a)
print("Common Frames: ", b)
video.release()
writer.release()
cv2.destroyAllWindows()

Total Frames:  2996
Unique Frames:  2945
Common Frames:  51


In [8]:
old = cv2.imread("img00001.jpg")
new = cv2.imread("img00002.jpg")


In [9]:
old.shape

(1080, 1920, 3)

In [6]:
p1, st, err = cv2.calcOpticalFlowPyrLK(old, new, None, 0.5, 3, 15, 3, 5, 1.2, 0)

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'calcOpticalFlowPyrLK'
> Overload resolution failed:
>  - Can't parse 'winSize'. Input argument doesn't provide sequence protocol
>  - Can't parse 'winSize'. Input argument doesn't provide sequence protocol


In [2]:
import numpy as np
import cv2

cap = cv2.VideoCapture('bern_4.mp4')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0,None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.5.5) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type


In [24]:
import cv2
import numpy as np
cap = cv2.VideoCapture("test_videos/Appenzell_4.mp4")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
a = []

ret, frame2 = cap.read()
nex = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

flow = cv2.calcOpticalFlowFarneback(prvs,nex, None, 0.5, 3, 15, 3, 5, 1.2, 0)
mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    

#print(flow[2])
print(flow.shape)
print(np.max(mag))
print(np.max(ang))

cap.release()
cv2.destroyAllWindows()

(1080, 1920, 2)
40.196346
6.2831855


In [9]:
flow = cv2.calcOpticalFlowFarneback(old,new, None, 0.5, 3, 15, 3, 5, 1.2, 0)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\video\src\optflowgf.cpp:1116: error: (-215:Assertion failed) prev0.size() == next0.size() && prev0.channels() == next0.channels() && prev0.channels() == 1 && pyrScale_ < 1 in function 'cv::`anonymous-namespace'::FarnebackOpticalFlowImpl::calc'


In [1]:
import cv2
import numpy as np
cap = cv2.VideoCapture("bern_4.mp4")

ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    cv2.imshow('frame2',rgb)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',rgb)
    prvs = next

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 